In [1]:
import sys, os, re, json, time

import pandas as pd
import pickle
import h5py

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import plotting
from PIL import Image
from tqdm import tqdm
from utils import imread, img_data_2_mini_batch, imgs2batch

from sklearn import metrics
from sklearn.metrics import accuracy_score

from naive import Enc, Dec, EncDec
from data_loader import VQADataSet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import transforms

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
N = 2000
dataset_filename = "./data/data_{}.pkl".format(N)
dataset = None
print(dataset_filename)
if (os.path.exists(dataset_filename)):
    with open(dataset_filename, 'rb') as handle:
        print("reading from " + dataset_filename)
        dataset = pickle.load(handle)
else:
    dataset = VQADataSet(Q=N)
    with open(dataset_filename, 'wb') as handle:
        print("writing to " + dataset_filename)
        pickle.dump(dataset, handle)

assert(dataset is not None)
def debug(v,q,a):
    print('\nV: {}\nQ: {}\nA: {}'.format(v.shape, q.shape, a.shape))


./data/data_2000.pkl
reading from ./data/data_2000.pkl


In [3]:
embed_size        = 128
hidden_size       = 128
batch_size        = 40
ques_vocab_size   = len(dataset.vocab['question'])
ans_vocab_size    = len(dataset.vocab['answer'])
rnn_layers        = 1
n_epochs          = 30
learning_rate     = 0.001
momentum          = 0.98

print(ques_vocab_size, ans_vocab_size)

1469 509


In [6]:
def eval_model(data_loader, model, criterion, optimizer, batch_size, training=False,
              epoch = 0, total_loss_over_epochs=[], scores_over_epochs=[]):
    running_loss = 0.
    final_labels, final_preds = [], []
    scores, losses = [], []
    if data_loader is None:
        return
    
    run_type = None
    if training:
        run_type = 'train'
        model.train()
    else:
        run_type = 'test'
        model.eval()
    
    for i, minibatch in enumerate(data_loader):
        # extract minibatch
        t0 = time.time()
        idxs, v, q, a, q_len = minibatch
        
        # convert torch's DataLoader output to proper format.
        # torch gives a List[Tensor_1, ... ] where tensor has been transposed. 
        # batchify transposes back.`
        v = v.to(device)
        q = VQADataSet.batchify_questions(q).to(device)
        a = a.to(device)

        logits = model(v, q, q_len)
        preds = torch.argmax(logits, dim=1)

#         loss = criterion(logits, a)
        loss = F.nll_loss(logits, a)
        running_loss += loss.item()
        
#         score = metrics.precision_recall_fscore_support(preds.tolist(),
#                                                         a.tolist(),
#                                                         average='weighted')
        score = metrics.accuracy_score(preds.tolist(),a.tolist())
    
        scores.append(score)
        losses.append(loss)
        
        loss_key = '{}_loss'.format(run_type)
        total_loss_over_epochs['{}_loss'.format(run_type)].append(loss)
        scores_over_epochs['{}_scores'.format(run_type)].append(score)
        
        if training and optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
   
        final_labels += a.tolist()
        final_preds  += preds.tolist()
        if i%10==0:
            score = np.mean(scores)
            print("Epoch {}: {} Loss: {} Score: {} t: {}".format(epoch, run_type,loss, score, time.time()-t0))
#             plotting.plot_score_over_n_epochs(scores_over_epochs, score_type='precision', fig_size=(7,3))
#             plotting.plot_loss_over_n_epochs(total_loss_over_epochs, hard_key=loss_key, fig_size=(7, 3))
            
    return running_loss, final_labels, final_preds

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = EncDec(embed_size, hidden_size, ques_vocab_size, ans_vocab_size, rnn_layers).to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.get_parameters(), lr=learning_rate, momentum=momentum)
optimizer = torch.optim.Adam(model.get_parameters(), lr=learning_rate)

train_loader = dataset.build_data_loader(train=True, args={'batch_size': batch_size})
test_loader  = dataset.build_data_loader(test=True, args={'batch_size': batch_size})

best_score = 0

train_all_loss, train_all_labels, train_all_preds = [], [], []
print("model built, start training.")
total_loss_over_epochs, scores_over_epochs = plotting.get_empty_stat_over_n_epoch_dictionaries()
total_loss_over_epochs2, scores_over_epochs2 = plotting.get_empty_stat_over_n_epoch_dictionaries()
for epoch in tqdm(range(n_epochs)):
    t0= time.time()
    tr_loss, tr_labels, tr_preds = eval_model(data_loader = train_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = optimizer,
                                     batch_size  = batch_size,
                                     training    = True,
                                     epoch       = epoch,
                                     total_loss_over_epochs = total_loss_over_epochs,
                                     scores_over_epochs     = scores_over_epochs)
    
    tr_loss, ts_labels, ts_preds = eval_model(data_loader = test_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = None,
                                     batch_size  = batch_size,
                                     training    = False,
                                     epoch       = epoch,
                                     total_loss_over_epochs = total_loss_over_epochs2,
                                     scores_over_epochs     = scores_over_epochs2)
    
    
    score = metrics.accuracy_score(ts_preds,ts_labels)
#     total_loss_over_epochs['train_loss'].append(tr_loss)
#     scores_over_epochs['train_scores'].append(train_scores)
    
#     if True:# or epoch%1 == 0:
    print("\n"+"#==#"*7 + "epoch: {}".format(epoch) + "#==#"*7)
    print('TEST ACC: {}'.format(score))
    print("#==#"*7 + "time: {}".format(time.time()-t0) + "#==#"*7 + "\n")
#         print(train_scores)
#     plotting.plot_score_over_n_epochs(scores_over_epochs, score_type='precision', fig_size=(8,5))
#     plotting.plot_loss_over_n_epochs(total_loss_over_epochs, fig_size=(8, 5), title="Loss")
    
    
    
    

  0%|          | 0/30 [00:00<?, ?it/s]

batch_size: 40 shuffle: True
batch_size: 40 shuffle: False
model built, start training.
Epoch 0: train Loss: 6.206993103027344 Score: 0.0 t: 0.32837677001953125
Epoch 0: train Loss: 5.847630500793457 Score: 0.14772727272727268 t: 0.3098018169403076
Epoch 0: train Loss: 5.083428859710693 Score: 0.1642857142857143 t: 0.3006899356842041
Epoch 0: train Loss: 4.00607967376709 Score: 0.1693548387096774 t: 0.3054647445678711
Epoch 0: train Loss: 5.201992988586426 Score: 0.17378048780487804 t: 0.30841994285583496
Epoch 0: test Loss: 1.6587327718734741 Score: 0.425 t: 0.28287696838378906


  3%|▎         | 1/30 [00:47<22:46, 47.12s/it]


#==##==##==##==##==##==##==#epoch: 0#==##==##==##==##==##==##==#
TEST ACC: 0.29133858267716534
#==##==##==##==##==##==##==#time: 47.11704778671265#==##==##==##==##==##==##==#

Epoch 1: train Loss: 5.721721172332764 Score: 0.175 t: 0.30323314666748047
Epoch 1: train Loss: 4.4188666343688965 Score: 0.19545454545454544 t: 0.2982058525085449
Epoch 1: train Loss: 5.051380634307861 Score: 0.1952380952380952 t: 0.30435895919799805
Epoch 1: train Loss: 4.466466903686523 Score: 0.18790322580645155 t: 0.3062562942504883
Epoch 1: train Loss: 4.478691577911377 Score: 0.19329268292682927 t: 0.30958056449890137
Epoch 1: test Loss: 1.572569727897644 Score: 0.475 t: 0.2927532196044922


  7%|▋         | 2/30 [01:32<21:46, 46.66s/it]

...


#==##==##==##==##==##==##==#epoch: 27#==##==##==##==##==##==##==#
TEST ACC: 0.4251968503937008
#==##==##==##==##==##==##==#time: 42.52508854866028#==##==##==##==##==##==##==#

Epoch 28: train Loss: 1.6109806299209595 Score: 0.65 t: 0.30482029914855957
Epoch 28: train Loss: 1.5844597816467285 Score: 0.7454545454545456 t: 0.3199646472930908
Epoch 28: train Loss: 1.436797022819519 Score: 0.7297619047619047 t: 0.30010342597961426
Epoch 28: train Loss: 1.8538010120391846 Score: 0.7241935483870967 t: 0.3082096576690674
Epoch 28: train Loss: 1.0410888195037842 Score: 0.7323170731707317 t: 0.3179192543029785
Epoch 28: test Loss: 1.4596585035324097 Score: 0.55 t: 0.2857050895690918


 97%|█████████▋| 29/30 [21:56<00:46, 46.83s/it]


#==##==##==##==##==##==##==#epoch: 28#==##==##==##==##==##==##==#
TEST ACC: 0.42913385826771655
#==##==##==##==##==##==##==#time: 48.08460855484009#==##==##==##==##==##==##==#

Epoch 29: train Loss: 1.0522981882095337 Score: 0.775 t: 0.31890010833740234
Epoch 29: train Loss: 1.329848051071167 Score: 0.7590909090909093 t: 0.3062753677368164
Epoch 29: train Loss: 0.8107045888900757 Score: 0.7761904761904762 t: 0.334261417388916
Epoch 29: train Loss: 1.2955089807510376 Score: 0.7637096774193549 t: 0.3094182014465332
Epoch 29: train Loss: 1.155544638633728 Score: 0.7615853658536584 t: 0.3137216567993164
Epoch 29: test Loss: 1.233087182044983 Score: 0.6 t: 0.28357481956481934


100%|██████████| 30/30 [22:41<00:00, 46.38s/it]


#==##==##==##==##==##==##==#epoch: 29#==##==##==##==##==##==##==#
TEST ACC: 0.4409448818897638
#==##==##==##==##==##==##==#time: 45.33402466773987#==##==##==##==##==##==##==#



In [ ]:
for epoch in range(1):
    ts_loss, ts_labels, ts_preds = eval_model(data_loader = test_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = None,
                                     batch_size  = batch_size,
                                     training    = False,
                                     epoch       = epoch,
                                     total_loss_over_epochs = total_loss_over_epochs2,
                                     scores_over_epochs     = scores_over_epochs2)
    score = metrics.accuracy_score(ts_preds,ts_labels)
    print("ACC: " + str(score))

In [ ]:
print(tr_labels[0])
print(tr_preds[0])